# Исследование надежности заемщиков.

**Цель проекта** — проверить две гипотезы:

* Семейное положение заемщика влияет на факт погашения погашения кредита
* Количество детей у заемщика влияет на факт погашения погашения кредита

Сначала проверим данные на ошибки, пропуски и дубликаты. Проведем предобработку, затем проверим гипотезы и ответим на вопросы.

## Шаг 1. Обзор данных

In [1]:
# импорт необходимых для работы библиотек
import pandas as pd
import os 
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('russian')

In [2]:
if os.path.exists('datasets/data.csv'):
    df = pd.read_csv('datasets/data.csv')
else:
    df = pd.read_csv('./datasets/data.csv')
# читаем данные
df.head(10) # выводим первые 10 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info() # общая информация по данным

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Описание данных
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Названия колонок соответствуют стилю.

Видно, что есть явные пропуски в столбцах `days_employed`, `total_income`. Количество пропусков в этих столбцах одинаковое.

Также видно, что в столбце `education` записи в разном регистре.

**Вывод:**

Чтобы продолжить исследование, нужно подготовить данные. В них есть проблемы:

* В двух столбцах датафрейма есть пропущенные значения. Их нужно как то заполнить или удалить.
* Столбцы типа `object` нужно проверить на явные и неявные дубликаты, привести к нормальному виду.

## Шаг 2.1 Заполнение пропусков

Заполним пропуски в колонке `total_income`.

In [4]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Видим, что во всех видимых строчках, где `total_income` равен `NaN`, `days_employed` тоже равен `NaN`. Проверим, везде ли так.

In [5]:
# Смотрим на уникальные значения столбца days_emploed, где total_income равен NaN
df[df['total_income'].isna()]['days_employed'].value_counts()

Series([], Name: days_employed, dtype: int64)

Действительно, это так. У людей, с пропущенным суммарным доходом отсутствуют значения отработанных дней.

Теперь проверим долю пропущенных значений в колонке `total_income` от всех значений.

In [6]:
# количество пропущенных значений делим на общее количество значений
df[df['total_income'].isna()].shape[0] / df['total_income'].shape[0]

0.10099883855981417

Примерно 10% данных в `total_income` - пропуски

Возможные причины пропуска могут быть связаны с тем, что человек не имел задекларированного дохода, работал неофициально. Причина пропуска не напоминает технический характер. Пропуск в этом столбце можно охарактеризовать как **случайный**.

**Заполним пропуски в `total_income` медианой, пропуски в `days_employed` нулями.**

Заполнять будем именно медианным значением, так как оно описывает данные более объективно и меньше подвержено выбросам, чем среднее значение.

In [7]:
df['total_income'].median() # медиана

145017.93753253992

In [8]:
# заменяем медианой пропущенные значения 'total_income'
df['total_income'] = df['total_income'].fillna(df['total_income'].median())

In [9]:
# заменяем 0 пропущенные значения 'days_employed'
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())

Проверим, остались ли пропуски:

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Шаг 2.2 Проверка данных на аномалии и исправления.

Посмотрим на общие статистики, чтобы понять, какие есть аномалии

In [11]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Сразу бросается в глаза две проблемы:**

* минимальное значение `children` равно -1 (максимальное 20, что тоже вызывает подозрения.)
* показатели `days_employed` отрицательные, а максимальное значение равно ~1100 годам

И еще стоит проверить значения `dob_years`, которые равны 0

### 2.2.1 days_employed

В колонке `days_employed` аномалия. Трудовой стаж не может быть отрицательным. Посмотрим на значения, которые больше нуля.

In [12]:
df[df['days_employed'] > 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


In [13]:
df[df['days_employed'] > 0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [14]:
df[(df['days_employed'] > 0) & (df['income_type'] != 'пенсионер')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


Видим, что все, кто работал положительное число дней являются пенсионерами, кроме двух безработных людей.

In [15]:
df[df['days_employed'] > 0].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,3445.000000,3445.000000,3445.000000,3445.000000,3445.000000,3445.000000,3445.000000
mean,0.128302,365004.309916,59.124819,0.914659,0.984325,0.052830,137124.105624
std,0.955042,21075.016396,7.580584,0.517103,1.316071,0.223727,80242.210917
min,-1.000000,328728.720605,0.000000,0.000000,0.000000,0.000000,20667.263793
25%,0.000000,346639.413916,56.000000,1.000000,0.000000,0.000000,82876.335652
50%,0.000000,365213.306266,60.000000,1.000000,0.000000,0.000000,118514.486412
75%,0.000000,383246.444219,64.000000,1.000000,2.000000,0.000000,169746.263276
max,20.000000,401755.400475,74.000000,4.000000,4.000000,1.000000,735103.270167


Смотря на интервал минимальных и максимальных значений по данной выборке можно предположить, что людям, которые находятся на пенсии, присваивали большое число отработанных дней. Двое безработных, которые попали в этот список не могли столько отработать. Их будем считать за выбросы.

In [16]:
df[df.days_employed < 0].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,18080.000000,18080.000000,18080.000000,18080.000000,18080.000000,18080.000000,1.808000e+04
mean,0.617146,-2214.778582,40.276825,0.798673,0.970299,0.086228,1.705014e+05
std,1.435446,2193.374054,10.974709,0.551921,1.439355,0.280708,9.998975e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.136765e+04
25%,0.000000,-2895.867863,32.000000,0.000000,0.000000,0.000000,1.135424e+05
50%,0.000000,-1347.906788,40.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-868.137150,48.000000,1.000000,1.000000,0.000000,1.998261e+05
max,20.000000,-24.141633,75.000000,4.000000,4.000000,1.000000,2.265604e+06


У людей с отрицательными значениями всё выглядит приемлимо. Минимальное значение взятое по модулю 18388 дня примерно равно 50 годам. А максимальное по модулю 24 дня.

In [17]:
df[df.days_employed < 0].sort_values('days_employed').head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
3974,0,-15835.725775,64,среднее,1,гражданский брак,1,F,компаньон,0,96858.531436,сыграть свадьбу
1539,0,-15785.678893,59,высшее,0,Не женат / не замужем,4,F,сотрудник,0,119563.851852,операции с коммерческой недвижимостью
4321,0,-15773.061335,61,среднее,1,гражданский брак,1,F,сотрудник,0,205868.585780,свадьба
7731,0,-15618.063786,64,среднее,1,женат / замужем,0,F,компаньон,0,296525.358574,высшее образование
15675,0,-15410.040779,65,высшее,0,женат / замужем,0,F,сотрудник,0,188800.068859,покупка жилой недвижимости


Люди, которые отработали больше всего времени из возрастной категории 55+. Ещё раз убеждаемся в том, что данные похожи на реальные.

**Аномалию устраним так:**

* Людям, которые в положительном диапазоне, заполним нулями. Это явные выбросы, они нам будут мешать.
* Людям, кто отработал отрицательное количество времени, присвоим положительный знак.

In [18]:
# заполняем нулями аномальные значения
df.loc[df['days_employed'] > 0, 'days_employed'] = 0

In [19]:
# проверяем
df[df.days_employed > 0].value_counts()

Series([], dtype: int64)

In [20]:
# заменяем знак минус на знак плюс - переписываем число, взяв его по модулю
df.loc[df['days_employed'] < 0, 'days_employed'] = df.loc[df['days_employed'] < 0, 'days_employed'].apply(lambda num : abs(num))


In [21]:
#проверяем
df.days_employed.describe()

count    21525.000000
mean      1860.311116
std       2168.026409
min          0.000000
25%        385.106616
50%       1203.369529
75%       2518.168900
max      18388.949901
Name: days_employed, dtype: float64

### 2.2.2 children

Минимальное значение children равно -1 (максимальное 20, что тоже вызывает подозрения.)

In [22]:
# список уникальных значений колонки children
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что есть определенный ряд количества детей: 0, 1, 2, 3, 4, 5. Значения -1, 20 из этого ряда явно выделяются.

20 детей - выбросы, либо это ошибка с вводом данных, где случайно добавили 0. Данные о доходах тоже никак не позволяют судить о том, что это не выбросы.

Сравнение статистик, где 20 и 2 детей соответственно.

In [23]:
df[df.children == 20].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,76.0,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,20.0,1836.917156,41.815789,0.842105,0.815789,0.105263,165913.640682
std,0.0,2174.894370,12.065058,0.433671,1.363432,0.308931,76236.454726
min,20.0,0.000000,0.000000,0.000000,0.000000,0.000000,53971.576721
25%,20.0,516.518947,33.750000,1.000000,0.000000,0.000000,120986.388504
50%,20.0,1202.884531,41.500000,1.000000,0.000000,0.000000,145017.937533
75%,20.0,2383.652717,50.000000,1.000000,1.000000,0.000000,203490.401462
max,20.0,11937.171389,69.000000,2.000000,4.000000,1.000000,441721.334145


In [24]:
df[df.children == 2].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2055.0,2055.000000,2055.000000,2055.000000,2055.000000,2055.000000,2.055000e+03
mean,2.0,2020.072429,35.770316,0.788808,0.451095,0.094404,1.691884e+05
std,0.0,1739.787240,7.221411,0.556603,0.970559,0.292461,1.020834e+05
min,2.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.809288e+04
25%,2.0,850.792503,31.000000,0.000000,0.000000,0.000000,1.069220e+05
50%,2.0,1407.831353,35.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,2.0,2717.388523,40.000000,1.000000,1.000000,0.000000,1.979646e+05
max,2.0,13039.072024,64.000000,3.000000,4.000000,1.000000,1.103455e+06


In [25]:
(76 / (2055 + 76)) * 100

3.5664007508212108

Показатели примерно похожи. Количество таких значений составляет 3.6% от всех данных с детьми равными 2 + 20.

Изменим количество детей с 20 на 2.

In [26]:
df.loc[df.children == 20, 'children'] = 2

Теперь разберемся со строками, где значение равно -1

In [27]:
df[df.children == -1].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,47.0,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,-1.0,1978.514580,42.574468,0.829787,0.957447,0.021277,153298.588506
std,0.0,2104.738732,11.357328,0.433346,1.428856,0.145865,67243.248222
min,-1.0,0.000000,23.000000,0.000000,0.000000,0.000000,36052.447435
25%,-1.0,492.090106,33.500000,1.000000,0.000000,0.000000,114252.758940
50%,-1.0,1203.369529,41.000000,1.000000,0.000000,0.000000,145017.937533
75%,-1.0,2970.960064,50.500000,1.000000,2.000000,0.000000,170162.421662
max,-1.0,9851.184337,69.000000,2.000000,4.000000,1.000000,321603.700207


In [28]:
(47 / (4818 + 47)) * 100

0.9660842754367934

Тоже напоминает техническую ошибку. Добавили - к 1 и получилось -1.

Так как количество таких значений составляет меньше 1% от всех данных с детьми равными 1 + (-1), присвоим 1 вместо -1.

In [29]:
df.loc[df.children == -1, 'children'] = 1

Проверим уникальные значения

In [30]:
df.children.value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### 2.2.3 dob_years == 0

In [31]:
df.loc[df['dob_years'] <= 18, ['dob_years']].value_counts()

dob_years
0            101
dtype: int64

В промежутке от 0 до 18 лет встречаются только значения 0 лет. Заполним их медианным значением возраста.

In [32]:
df[df['dob_years'] != 0]['dob_years'].median() # медиана не учитывая 0

43.0

In [33]:
# заменяем значения
df.loc[df['dob_years'] == 0, 'dob_years'] = df[df['dob_years'] != 0]['dob_years'].median()

In [34]:
# проверяем что нули ушли
df.loc[df['dob_years'] < 18, ['dob_years']].value_counts()

Series([], dtype: int64)

## Шаг 2.3. Изменение типов данных.

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Заменим тип данных `float64` колонки `total_income` на `int`.

In [36]:
df['total_income'] = df['total_income'].astype('int')

In [37]:
# проверим изменения
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


## Шаг 2.4. Удаление дубликатов.

Сначала удалим явные дубликаты. Так как у нас нет колонки `id` мы не сможем судить точно дубликаты ли это или нет. Но предлагаю всё равно их удалить, так как они занимают маленькую часть данных и их отсутствие не сильно скажется на анализе.

In [38]:
# количество дубликатов
df.duplicated().sum()

54

In [39]:
# удалим дубликаты
df = df.drop_duplicates().reset_index(drop=True)

In [40]:
# проверяем
df.duplicated().sum()

0

Теперь проверим неявные

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     21471 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      21471 non-null  int64  
 11  purpose           21471 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Неявные дубликаты могут быть в колонках `education`, `family_status`, `gender`, `income_type`, `purpose`

In [42]:
# смотрим на уникальные значения
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [43]:
# исправляем, сведя все к маленьким буквам
df['education'] = df['education'].str.lower()

In [44]:
# проверяем
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [45]:
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

В колонке `family_status` всё хорошо. Исправление не требуется

In [46]:
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

 Тут тоже все хорошо. `XNA` - intersex гендер.

In [47]:
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Тут тоже всё оки.

In [48]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Выделим основные группы колонки `purpose`:
* Свадьба
* Жилье, недвижимость
* Автомобиль
* Образование

Это пригодится нам в [шаге 2.7](#step2_7)

In [49]:
# функция, которая разделит все значения колонки `purpose` на 4
def categorize_by_keywords(arg):
    stemmed_query = stemmer.stem(arg)
    for word in stemmed_query.split(' '):
        if word == 'жил' or word == 'недвижим' or word == 'жилья':
            return 'операции с недвижимостью'
        elif word == 'автомобил' or word == 'авто' or word == 'автомобили' or word == 'автомоб':
            return 'операции с автомобилем'
        elif word == 'образован':
            return 'получение образования'
        elif word == 'свадьб':
            return 'проведение свадьбы'

In [50]:
# заменим все значения на определенные
df['purpose'] = df['purpose'].apply(categorize_by_keywords)

In [51]:
df['purpose'].isna().sum()

0

In [52]:
# проверим
df['purpose'].value_counts()

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose, dtype: int64

**Вывод:**
В колонке `education` все значения уже были одинаковые, нужно было только привести их к единому регистру. Возможные причины этому могли быть неаккуратность введения данных. Кто то пишет с большой буквы, кто то с маленькой, кто то капсом.

В колонке `purpose` пришлось применить стемминг, чтобы выделить корни слов и по ключевым словам категоризировать цели взятия кредита. Такое происходит из-за того, что люди, которые заполняют колонку делают это разными словами, как придется.

## Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма с уникальными значениями по `education, education_id` и `family_status, family_status_id`

In [53]:
education_df = pd.DataFrame(df.loc[:, 'education':'education_id']).drop_duplicates().set_index('education_id')
family_status_df = pd.DataFrame(df.loc[:, 'family_status':'family_status_id']).drop_duplicates().set_index('family_status_id')


Проверим

In [54]:
education_df

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


In [55]:
family_status_df

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


Теперь удалим колонки `education` и `family_status` из основного датафрейма

In [56]:
# удаляем
df = df.drop(['education', 'family_status'], axis=1)

In [57]:
# проверяем
df.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

## Шаг 2.6. Категоризация дохода.

Создадим новый столбец `total_income_category` на основании столбца `total_income` следующим образом:

* 0–30000 — `E`;
* 30001–50000 — `D`;
* 50001–200000 — `C`;
* 200001–1000000 — `B`;
* 1000001 и выше — `A`.

In [58]:
df.total_income.describe()

count    2.147100e+04
mean     1.652096e+05
std      9.798386e+04
min      2.066700e+04
25%      1.076545e+05
50%      1.450170e+05
75%      1.957515e+05
max      2.265604e+06
Name: total_income, dtype: float64

Все значения больше 0.

Напишем функцию, которая будет принимать число и возвращать строчку в зависимости от категории, в которое число попало.

In [59]:
def total_income_categoriser(arg):
    if arg > 0 and arg <= 30000:
        return 'E'
    elif arg >= 30001 and arg <= 50000:
        return 'D'
    elif arg >= 50001 and arg <= 200000:
        return 'C'
    elif arg >= 200001and arg <= 1000000:
        return 'B'
    else:
        return 'A'

In [60]:
# создадим новый столбец, применив функцию total_income_categoriser
df['total_income_category'] = df['total_income'].apply(total_income_categoriser)

In [61]:
# проверяем
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,операции с недвижимостью,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,получение образования,B
4,0,0.000000,53,1,1,F,пенсионер,0,158616,проведение свадьбы,C


## Шаг 2.7. Категоризация целей кредита. <a id='step2_7'></a>

Так как мы уже категоризировали колонку `purpose` ранее в значения:
* операции с автомобилем,
* операции с недвижимостью,
* проведение свадьбы,
* получение образования.

Просто изменим название колонки `purpose` на `purpose_category`.

In [62]:
df = df.rename(columns={'purpose': 'purpose_category'}, errors='raise')

In [63]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_category,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,операции с недвижимостью,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,получение образования,B
4,0,0.000000,53,1,1,F,пенсионер,0,158616,проведение свадьбы,C


## Ответы на вопросы.

#### Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок?

`debt` — имел ли задолженность по возврату кредитов

In [64]:
debt_children = df.pivot_table(index='children', columns= 'debt', values='days_employed', aggfunc='count')

debt_children['ratio_%'] = (debt_children[1] / (debt_children[0] + debt_children[1])) * 100
debt_children

debt,0,1,ratio_%
children,,,
0,13044.0,1063.0,7.535266
1,4411.0,445.0,9.163921
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


#### Вывод 1:

Если у ребенка 0 детей, то он наиболее вероятно вернет кредит. Если у ребенка 4 ребенка, то он наименее вероятно вернет кредит.

Данные не позволяют четко утверждать о зависимости возврата кредита и детей, но можно предположить, что чем больше в семье детей, тем менее вероятен возврат кредита.

#### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [65]:
family_status_debt = df.join(family_status_df, on='family_status_id', how='left') \
                       .pivot_table(index='family_status', columns='debt', values='gender', aggfunc='count')
family_status_debt['ratio_%'] = (family_status_debt[1] / (family_status_debt[1] + family_status_debt[0])) * 100

In [66]:
family_status_debt

debt,0,1,ratio_%
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3775,388,9.320202
женат / замужем,11413,931,7.542126


#### Вывод 2:

Люди со статусами `Не женат / не замужем`, `гражданский брак` наименее вероятно возвращали кредиты. Люди со статусом `женат / замужем`, `вдовец / вдова`, `в разводе` наиболее вероятно возвращали кредиты.

Делая такой вывод, следует помнить о том, что количество данных в группах довольно сильно различаются и такой прогноз может быть не точным.

#### Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [67]:
total_income_debt = df.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count')
total_income_debt['ratio_%'] = (total_income_debt[1] / (total_income_debt[1] + total_income_debt[0])) * 100

In [68]:
total_income_debt

debt,0,1,ratio_%
total_income_category,,,
A,23,2,8.000000
B,4685,356,7.062091
C,14673,1360,8.482505
D,329,21,6.000000
E,20,2,9.090909


#### Вывод 3:

Заработок напоминает нормальное распределение. Больше всего людей со средним заработком, меньше всего тех, кто зарабатывает очень много или совсем не зарабатывает.

Судить о том, кто наиболее вероятно вернет кредит сложно, но можно отметить, что наименьшая доля тех, кто не вернул кредит отмечается у людей с категориями зароботка `D` и `B`.

#### Вопрос 4:
Как разные цели кредита влияют на его возврат в срок?


In [69]:
purpose_category_debt = df.pivot_table(index='purpose_category', columns='debt', values='gender', aggfunc='count')
purpose_category_debt['ratio_%'] = (purpose_category_debt[1] / (purpose_category_debt[1] + purpose_category_debt[0])) * 100

In [70]:
purpose_category_debt

debt,0,1,ratio_%
purpose_category,,,
операции с автомобилем,3905,403,9.354689
операции с недвижимостью,10032,782,7.231367
получение образования,3644,370,9.217738
проведение свадьбы,2149,186,7.965739


#### Вывод 4

Люди которые берут кредит на `операции с недвижимостью` и `проведение свадьбы` наиболее вероятно вернут кредит, чем те, кто берет кредит на `операции с автомобилем` и `получение образования`. 

# Общий вывод:

Было обнаружено много ошибок в данных. Все они были исправлены и очищены. 

Предобработанные данные можно интерпретировать так:
Если у человека нет детей, он женат / замужем, вдовец / вдова, в разводе, и он берет кредит на недвижимость или свадьбу, то вероятность того, что он вернет кредит - наибольшая.